In [1]:
import pandas as pd
import numpy as np
import pathlib
from ast import literal_eval
import graph_tool as gt
from graph_tool.all import graph_draw


In [2]:
# df = pd.read_csv("../../data/sample_data.csv",)
# df = pd.read_csv("../../data/first_14_days_UTXO_txs_of_IOTA.csv")
df = pd.read_csv("../../data/chunk.csv")
# df.drop(['message_id', 'milestone_index', 'datetime'], axis=1, inplace=True)
df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'block_index', 'timestamp'], axis=1, inplace=True)


In [3]:
df_unique_tx_id = pd.DataFrame()
df_unique_tx_id['transaction_id'] = pd.unique(df[['transaction_id']].values.ravel())
df_unique_tx_id['tx_unique_id'] = df_unique_tx_id.index

pathlib.Path('../../logs/IoTa_logs/trial_files/').mkdir(parents=True, exist_ok=True)
df_unique_tx_id.to_csv('../../logs/IoTa_logs/trial_files/transaction_id.csv', index=False)


In [4]:
df = df.merge(df_unique_tx_id, on='transaction_id', how='left')
df.drop(['transaction_id'], axis=1, inplace=True)

print(df.shape)
df = df[df['input_addresses_x'].str.contains("\[nan|nan,|, nan|nan]") == False]
print(df.shape)


(10000000, 5)
(10000000, 5)


In [5]:
df


,input_addresses_x,input_amounts_x,output_addresses_y,output_amounts_y,tx_unique_id
0,['12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'],[5000000000],"['1Q2TWHE3GMdB6BZKafqwxXtWAWgFt5Jvm3','12cbQLT...","[1000000000,4000000000]",0
1,['12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'],[4000000000],"['1DUDsfc23Dv9sPMEk5RsrtfzCw5ofi5sVW','12cbQLT...","[1000000000,3000000000]",1
2,['12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'],[3000000000],"['1LzBzVqEeuQyjD2mRWHes3dgWrT9titxvq','12cbQLT...","[100000000,2900000000]",2
3,['12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'],[2900000000],"['13HtsYzne8xVPdGDnmJX8gHgBZerAfJGEf','12cbQLT...","[100000000,2800000000]",3
4,['13HtsYzne8xVPdGDnmJX8gHgBZerAfJGEf'],[100000000],['15NUwyBYrZcnUgTagsm1A7M2yL2GntpuaZ'],[100000000],4
...,...,...,...,...,...
9999995,['1HHoxHYLEsHpX1XLMufH2u22BnyjBakac5'],[65894581896],"['11D4FWQiGVGnCbvAMPvBZWBX7KMU2ZFsp','18VTbDeQ...","[65894131896,400000]",9999995
9999996,['19x9YXEtWgWSEnLJprnaRtUm4nSEc7cZoW'],[13384010000],"['1GgretSpZJ3BHzfesyFyCvMQeb5oN4NphF','1Bez92i...","[12384010000,1000000000]",9999996
9999997,['16DcDa7hNBCxNAug16PpXGYLpkzJvzq1mo'],[8010000000],"['1NBzzPyfbW7MZVsYpajrBVdbutHY3eyTrA','19wPrnM...","[8009460000,490000]",9999997
9999998,['1AqG4ECFDkm6W6AV3kEmJyFqBr4rCoeXVg'],[1000000000],"['1A7RGNN9oF4UNuoMEw42AqZeMFiXuqCAYP','14W1pPx...","[250000000,500000000,250000000]",9999998


In [6]:
def create_graph(addrs_list, graph):
    for a in literal_eval(addrs_list):
        print(a)
    # graph.add_vertex(literal_eval(addrs_list))

    return graph


In [7]:

# lst = ['a', 'b', 'c']
# res = map(int, lst)
# print(list(res))


In [8]:
# # df['input_addresses_x'].apply(eval)
# g = gt.Graph(directed=False)
# g = df['input_addresses_x'].apply(create_graph, graph = g)


In [9]:
graph_of_correspondences = gt.Graph( directed=False )
nuser_vertex = {}
vertex_nuser = graph_of_correspondences.new_vertex_property("string")
correspondences = None
counter = 0
n_users = 0


In [10]:
def normalise_ls(ls):

    ret = []
    ls = literal_eval(ls)
    if type(ls) == type(1) or type(ls) == np.int32:
        ret = [ls]
    elif type(ls) == list or type(ls) == set:
        ls2 = set()
        for i in ls:
            if type(i) == type(1) or type(i) == np.int32:
                ls2.add(i)
            elif type(i) == type(""):
                ls2.update(  map(np.int32, [_ for _ in ls]) )

        ret = list(ls2)

    return ret


In [11]:
# provide a list of addresses that belong to a user - ls
# useful def - adds needed edges
from cmath import isnan
from operator import lshift


def add_correspondence(ls, graph_of_correspondences):
    # ls = normalise_ls(ls)
    if 'nan' in ls: ls.replace('nan', '0')

    ls = set(literal_eval(ls)) # converts string to list form

    if '0' in ls: ls.remove('0')

    if len(ls) <= 1:
        return

    # ls = sorted( ls )

    for nu in ls :
        # if nuser_vertex.has_key(nu):
        if nu in nuser_vertex:
            continue
        v = graph_of_correspondences.add_vertex()
        idx = graph_of_correspondences.vertex_index[v]
        
        # can skip next two lines. every time a new node is added, it won't be mapped to the corrs. network automatically. so these two lines are for this mapping.
        
        nuser_vertex[nu] = idx
        vertex_nuser[ v ] = nu


    v0 = nuser_vertex[ls.pop()]


    for nu in ls:
        v1 = nuser_vertex[nu]
        if not graph_of_correspondences.edge(v0,v1):
            graph_of_correspondences.add_edge(v0,v1)
            # counter += 1
            # initialize edge weight to 1

        # else edge_weight + = 1 # existing edge so increase weight by 1
    
    return



In [12]:
df['input_addresses_x'].apply(add_correspondence, graph_of_correspondences = graph_of_correspondences)


0          None
1          None
2          None
3          None
4          None
           ... 
9999995    None
9999996    None
9999997    None
9999998    None
9999999    None
Name: input_addresses_x, Length: 10000000, dtype: object

In [13]:
graph_of_correspondences


<Graph object, undirected, with 4295966 vertices and 4435829 edges, at 0x1497a79a0>

In [14]:
# from graph_tool import draw
# draw.graph_draw(graph_of_correspondences)

# gt.GraphView(g)